In [1]:
import pandas as pd
import joblib

In [2]:
import os

clf = joblib.load("models/genre_mood_model.pkl")
vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
try:
    mlb_path = os.path.join("..", "data", "genre_mlb.pkl")
    mlb = joblib.load(mlb_path)
except FileNotFoundError:
    print(f"Warning: '{mlb_path}' not found. 'mlb' is not loaded.")
    mlb = None

In [5]:
genre_to_mood = {
    "Comedy": "Happy",
    "Romance": "Romantic",
    "Drama": "Thoughtful",
    "Horror": "Scared",
    "Adventure": "Excited",
    "Action": "Energetic",
    "Thriller": "Tense",
    "Sci-Fi": "Curious"
}

# Load your movies (from DB or another CSV)
my_movies = pd.read_csv("../data/test.movies.csv")  # Load from data folder
my_movies["overview"] = my_movies["overview"].fillna("")
X_my = vectorizer.transform(my_movies["title"] + " " + my_movies["overview"])

# Predict genres
y_pred = clf.predict(X_my)
pred_genres = mlb.inverse_transform(y_pred)

# Map to moods
my_movies["Predicted Genres"] = pred_genres
my_movies["Mapped Moods"] = my_movies["Predicted Genres"].apply(
    lambda g_list: list({genre_to_mood.get(g, "Neutral") for g in g_list})
)

print(my_movies[["title", "Predicted Genres", "Mapped Moods"]])

                                         title        Predicted Genres  \
0                            War of the Worlds         (Action, Drama)   
1                                     Superman      (Adventure, Drama)   
2    Mission: Impossible - The Final Reckoning     (Action, Adventure)   
3                              Striking Rescue      (Action, Thriller)   
4                       Jurassic World Rebirth     (Action, Adventure)   
..                                         ...                     ...   
591                Maze Runner: The Death Cure               (Action,)   
592                          Kill Bill: Vol. 1  (Action, Crime, Drama)   
593                       Boyka: Undisputed IV                      ()   
594                  The Princess and the Frog               (Comedy,)   
595              My Hero Academia: You're Next                (Drama,)   

                         Mapped Moods  
0             [Thoughtful, Energetic]  
1               [Excited, Thoug